In [1]:
#import statements, hyper params, models
import numpy as np
import sys
from sklearn.metrics import mean_squared_error
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from matplotlib.pyplot import figure
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore') 
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from numpy import random
from random import randrange
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import math
import time
    #.02 is a "good"
THRESH_HOLD=.01          #MSE value when we consider the second model to have "cracked" the first
EPOCHS = 5                 #number of passes of whole data
BATCH_SIZE =  64            #size of data going through at once
LEARNING_RATE = 0.01        #how much we shift parameters during back prop
NEURONS=25                   #number of nodes in each layer
INPUTS=5                   #number of input vars
XSIZE=  200                 #size of each data set generated
BAD_LOSS_PASSES=10000        #number of times we have small loss until we stop code
NO_TIME_INCREASES_STOP=60   #number of interations with no time increase until we stop code
TESTMODULE_LAYERS=5        #number of layers in cracking module
GROUND_TRUTH_LAYERS=5      #number of layers in ground thruth models
LINCOMBO=False               #wether to take best new time or linear combination
activation=nn.Sigmoid()     #activation  
SPARSE=False
def get_Xvals(datatype):
    if not SPARSE:
        return (np.random.uniform( size=(XSIZE, INPUTS))-0.5)*np.sqrt(12)
    else:
        sparsedata=(np.random.uniform( size=(XSIZE, INPUTS))-0.5)*np.sqrt(12)
        for i in sparsedata:
            for j in i:
                if np.random.randint(0,2)==0:
                    j=0
        for i in sparsedata:
            for j in i:
                if np.random.randint(0,2)==0:
                    j=0
        return sparsedata


                
    
def getdata_XY(datatype):
    X=get_Xvals(datatype)
    test_data = testData(torch.FloatTensor(X))
    test_loader = DataLoader(dataset=test_data, batch_size=1)
    y_truthlist =[]
    
    
    GroundTruth.eval()
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            y_test_pred = GroundTruth(X_batch)
            y_test_pred = torch.sigmoid(y_test_pred)
            y_truthlist.append(y_test_pred.cpu().numpy())


    y_truthlist = [a.squeeze().tolist() for a in y_truthlist]
    return X, y_truthlist

def get_yvals(model, X_data):
    model.eval()
    test_data = testData(torch.FloatTensor(X_data))
    test_loader = DataLoader(dataset=test_data, batch_size=1)
    y_truthlist=[]
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            y_test_pred = model(X_batch)
            y_test_pred = torch.sigmoid(y_test_pred)
            y_truthlist.append(y_test_pred.cpu().numpy())


    y_truthlist = [a.squeeze().tolist() for a in y_truthlist]
    return y_truthlist

#Data class
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

#Data loaders (to look at data) and deal with data (training and testing)
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

#trains a network, returns time spent training 
def trainnetwork(network, train_loader, optimizer, Time=True):
    t0=time.time()
    network.to(device)
    criterion = nn.MSELoss()


    network.train()
    for e in range(1, EPOCHS+1):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = network(X_batch)
            y_pred = torch.sigmoid(y_pred)
            loss = criterion(y_pred, y_batch.unsqueeze(1))
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()        
            
    t1=time.time()
    return(t1-t0)

#returns next ground truth model
#model obtained by taking average of models with better times than previous model 
def get_new_GroundTruth_lincombo(model_list, time_list, time):
    better_model_list=[]
    for (i, model) in zip(time_list, model_list):
        if i>time:
            better_model_list.append(model)
    
    newGroundTruth=GroundTruth.get_copy()
    
    layer_weight=torch.tensor([[0]*INPUTS]*NEURONS, requires_grad=False)
    for i in better_model_list:
        layer_weight=layer_weight+i.layer_input.weight/(len(better_model_list))
        
    layer_weight.requires_grad_()    
    newGroundTruth.layer_input.weight=torch.nn.Parameter(layer_weight)
    
    
    for i in range(GROUND_TRUTH_LAYERS):
        layer_weight=torch.tensor([[0]*NEURONS]*NEURONS, requires_grad=False)
        for bettermodel in better_model_list:
            layer_weight=layer_weight+bettermodel.layers[i].weight/(len(better_model_list))
        layer_weight.requires_grad_()
        newGroundTruth.layers[i].weight=torch.nn.Parameter(layer_weight)
        
    layer_weight=torch.tensor([0]*NEURONS, requires_grad=False)
    for i in better_model_list:
        layer_weight=layer_weight+i.layer_output.weight/(len(better_model_list))
    layer_weight.requires_grad_()   
    newGroundTruth.layer_output.weight=torch.nn.Parameter(layer_weight)

    return newGroundTruth
        
#checks accuracy of model
def check_threshhold(network, test_loader, truth):
    total_loss=0
    network.eval()
    with torch.no_grad():
        for (X_batch, y) in zip(test_loader, truth):
            X_batch = X_batch.to(device)
            y_test_pred = network(X_batch)
            y_pred_tag = torch.sigmoid(y_test_pred)
            total_loss=total_loss+((y_pred_tag.cpu().numpy()-y))**2
    
    total_loss=total_loss/len(truth)

    if total_loss[0][0]<THRESH_HOLD:
        return True, total_loss[0][0]
    else:
        return False, total_loss[0][0]
                       
def select_node():
    layer=randrange(GROUND_TRUTH_LAYERS+2)
    if layer==0:
        #node=[randrange(INPUTS), randrange(NEURONS-1)]
        layer=1
    if layer==GROUND_TRUTH_LAYERS+1:
         node=randrange(NEURONS-1)
    else:
        node=[randrange(NEURONS-1), randrange(NEURONS-1)]
    return [layer, node]
       
class arbitraryLayerModel(nn.Module):
    def __init__(self, num_layers):
        super(arbitraryLayerModel, self).__init__()
        self.layer_input=nn.Linear(INPUTS, NEURONS)
        self.layers=nn.ModuleList([nn.Linear(NEURONS, NEURONS) for i in range(num_layers)])
        
        
        self.batchnorms=nn.ModuleList([nn.BatchNorm1d(NEURONS) for i in range(num_layers)])
        self.batchnorm_input=nn.BatchNorm1d(NEURONS)
        
        self.layer_output=nn.Linear(NEURONS,30)
        
    def get_copy(self):
        copied_model=arbitraryLayerModel(len(self.layers))
        copied_model.layers=nn.ModuleList()
        for i in self.layers:
            copied_model.layers.append(copy.deepcopy(i))
            
        copied_model.batchnorms=nn.ModuleList()
        for i in self.batchnorms:
            copied_model.batchnorms.append(copy.deepcopy(i))
        copied_model.layer_input=copy.deepcopy(self.layer_input)
        copied_model.layer_output=copy.deepcopy(self.layer_output)
        copied_model.batchnorm_input=copy.deepcopy(self.batchnorm_input)

        return copied_model
            
     
    def forward(self, inputs):
        x=activation(self.batchnorm_input(self.layer_input(inputs)))
        
        for (layer, batchnorm) in zip(self.layers, self.batchnorms):
            x=activation(batchnorm(layer(x)))
                
                
        x=self.layer_output(x)
        return(x)

    
    
    def change_node_model(self, node):
        changed_node=self.get_copy()
        
        weightchange=np.random.normal(0,1)      
        
        if node[0]==0:
            zeros=[[0]*INPUTS]*NEURONS
            zeros[node[1][0]][node[1][1]]=weightchange
            newweight=self.layer_input.weight.requires_grad_(False)+torch.tensor(zeros,requires_grad=False)
            newweight.requires_grad_()
            changed_node.layer_input.weight=torch.nn.Parameter(newweight)    
            return changed_node
            
            
        
        if node[0]==GROUND_TRUTH_LAYERS+1:
            zeros=[0]*(NEURONS-1)
            zeros.insert(node[1],weightchange)
            newweight=self.layer_output.weight.requires_grad_(False)+torch.tensor(zeros,requires_grad=False)
            newweight.requires_grad_()
            changed_node.layer_output.weight=torch.nn.Parameter(newweight) 
            
            
        
        else:
            zeros = [[0.0]*NEURONS]*NEURONS
            for layer, i in enumerate(changed_node.layers):
                if i==node[0]+1:
                    zeros[node[1][0]][node[1][1]]=weightchange
                    newweight=layer.weight.requires_grad_(False)+torch.tensor(weightchange,requires_grad=False)
                    newweight.requires_grad_()
                    layer.weight=torch.nn.Parameter(newweight)
                    
        

        return changed_node

   
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")   

    

   

In [ ]:
#linear combo 1 weight guassian weight
       
#groundtruth model  
#GroundTruth = arbitraryLayerModel(GROUND_TRUTH_LAYERS)
possible_infinite_time=arbitraryLayerModel(GROUND_TRUTH_LAYERS)



firstmodeltrained=arbitraryLayerModel(TESTMODULE_LAYERS)
test_model_optimizer = optim.SGD(firstmodeltrained.parameters(), lr=LEARNING_RATE)
accepted=False
current_time=0
data_list=[]


while((not accepted)):
    X, y=getdata_XY(SPARSE)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
    test_data = testData(torch.FloatTensor(X_test))
    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)                      
    test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)
    data_list.append(X)
    current_time=current_time+trainnetwork(firstmodeltrained, train_loader,test_model_optimizer)
    accepted=check_threshhold(firstmodeltrained, test_loader, y_test)
    
print("First time: ", current_time)
    
#iterates
interationcounter=1
#number of iterates in a row with no time increase
no_increasecounter=0
time_increase=True
#number of training interations where loss doesnt change
loss_difference_counter=0
stop_itterations=False
while(time_increase):
    node_list=[]
    time_list=[]
    model_list=[]
    for i in range(0, 10):
        node_list.append(select_node())
    
    test_model_list=[arbitraryLayerModel(TESTMODULE_LAYERS) for i in range(10)]
    for i in range(0, 10):
        total_time=0
        infinite_time_counter=0
        prospective_GroundTruth=GroundTruth.change_node_model(node_list[i])

        model_list.append(prospective_GroundTruth)

        for j, test_model in enumerate(test_model_list):
            current_loss=0
            previous_loss=0
            train_testmodel=test_model.get_copy()
            test_model_optimizer = optim.SGD(train_testmodel.parameters(), lr=LEARNING_RATE)
            data_counter=0
            test_time=0
            inital_data_length=len(data_list)
            accepted=False
            loss_difference_counter=0
            Model_beaten=False

            while(not accepted): 
                #use previously generated data
                if data_counter<inital_data_length:
                    X=data_list[data_counter]
                    y=get_yvals(prospective_GroundTruth, X)
                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                    test_data = testData(torch.FloatTensor(X_test))
                    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                    test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)




                    test_time=test_time+trainnetwork(train_testmodel, train_loader,test_model_optimizer)
                    previous_loss=current_loss
                    accepted, current_loss=check_threshhold(train_testmodel, test_loader, y_test)
                    loss_diff=previous_loss-current_loss
                    data_counter=data_counter+1
                #generate new data if needed
                else:                          
                    X=get_Xvals(SPARSE)
                    data_list.append(X)
                    y=get_yvals(prospective_GroundTruth, X)



                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                    test_data = testData(torch.FloatTensor(X_test))
                    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                    test_loader = DataLoader(dataset=test_data, batch_size=1)

                    previous_loss=current_loss
                    test_time=test_time+trainnetwork(train_testmodel, train_loader,test_model_optimizer)
                    accepted, current_loss=check_threshhold(train_testmodel, test_loader, y_test)
                    loss_diff=previous_loss-current_loss


                #checks if loss difference is small
                
                if abs(loss_diff)<(THRESH_HOLD/10):
                    loss_difference_counter=loss_difference_counter+1
                    if loss_difference_counter>BAD_LOSS_PASSES:
                        infinite_time_counter=infinite_time_counter+1
                        #print("Test model beaten. Total beaten by prospective ground truth: ", infinite_time_counter)
                        Model_beaten=True
                    #elif loss_difference_counter>5:
                        #print(f'Number no loss change {loss_difference_counter}: | Current time: {test_time:.3f} | Loss difference: {abs(loss_diff):.4} | Loss: {current_loss:.4}')
 
            
                if Model_beaten:
                    break


            total_time=total_time+test_time

    
    
        print(f'Iteration: {interationcounter} | Ground truth number: {i+1} | Average time:  {total_time/10:.3}')

        if infinite_time_counter==10:
            print("Potential unbreakable found")
            possible_infinite_time=prospective_GroundTruth.get_copy()
            break
        time_list.append(total_time/(10))

        
           
    
    if infinite_time_counter==10:
        break
    max_new_time=max(time_list)
    
    
    
    #prints iteration data
    print(f'Iteration {interationcounter}: | Previous time: {current_time:.3f} | Best new time: {max_new_time:.3f} | Difference: {max_new_time-current_time:.3f}')
    interationcounter=interationcounter+1
    
    
    
     
    
    
    time_increase=max_new_time-current_time
    
    
    
    #changes ground truth if better time found
    if max_new_time>current_time and max_new_time<current_time*50:
        if LINCOMBO:
            GroundTruth=get_new_GroundTruth_lincombo(model_list, time_list, current_time)
        else:
            maxindex=time_list.index(max(time_list))
            GroundTruth=model_list[maxindex].get_copy()
        time_increase=True
        previous_time=current_time
        current_time=max_new_time
        no_increasecounter=0
        
    else:
        no_increasecounter=no_increasecounter+1

    #if no better ground truth found for 10 iterations, stops code                 
    if no_increasecounter==NO_TIME_INCREASES_STOP:
        print("No increase for ", NO_TIME_INCREASES_STOP,  " iterations")
    #    time_increase=False
 
      

                    

In [ ]:
#only 1 ground truth generated
time_for_graph=[]      
#groundtruth model  
GroundTruth = arbitraryLayerModel(GROUND_TRUTH_LAYERS)
#GroundTruth=possible_infinite_time.get_copy()
possible_infinite_time=arbitraryLayerModel(GROUND_TRUTH_LAYERS)



firstmodeltrained=arbitraryLayerModel(TESTMODULE_LAYERS)
test_model_optimizer = optim.SGD(firstmodeltrained.parameters(), lr=LEARNING_RATE)
accepted=False
current_time=0
data_list=[]


while((not accepted)):
    X, y=getdata_XY(SPARSE)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
    test_data = testData(torch.FloatTensor(X_test))
    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)                      
    test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)
    data_list.append(X)
    current_time=current_time+trainnetwork(firstmodeltrained, train_loader,test_model_optimizer)
    accepted=check_threshhold(firstmodeltrained, test_loader, y_test)
    
print("First time: ", current_time)
    
#iterates
interationcounter=1
#number of iterates in a row with no time increase
no_increasecounter=0
time_increase=True
#number of training interations where loss doesnt change
loss_difference_counter=0
stop_itterations=False
while(time_increase):
    node_list=[]
    time_list=[]
    model_list=[]
    for i in range(0, 10):
        node_list.append(select_node())
    
    test_model_list=[arbitraryLayerModel(TESTMODULE_LAYERS) for i in range(10)]
    for i in range(0, 1):
        total_time=0
        infinite_time_counter=0
        prospective_GroundTruth=GroundTruth.change_node_model(node_list[i])

        model_list.append(prospective_GroundTruth)

        for j, test_model in enumerate(test_model_list):
            current_loss=0
            previous_loss=0
            train_testmodel=test_model.get_copy()
            test_model_optimizer = optim.SGD(train_testmodel.parameters(), lr=LEARNING_RATE)
            data_counter=0
            test_time=0
            inital_data_length=len(data_list)
            accepted=False
            loss_difference_counter=0
            Model_beaten=False

            while(not accepted): 
                #use previously generated data
                if data_counter<inital_data_length:
                    X=data_list[data_counter]
                    y=get_yvals(prospective_GroundTruth, X)
                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                    test_data = testData(torch.FloatTensor(X_test))
                    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                    test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)




                    test_time=test_time+trainnetwork(train_testmodel, train_loader,test_model_optimizer)
                    previous_loss=current_loss
                    accepted, current_loss=check_threshhold(train_testmodel, test_loader, y_test)
                    loss_diff=abs(previous_loss-current_loss)
                    data_counter=data_counter+1
                #generate new data if needed
                else:                          
                    X=get_Xvals(SPARSE)
                    data_list.append(X)
                    y=get_yvals(prospective_GroundTruth, X)



                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                    test_data = testData(torch.FloatTensor(X_test))
                    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                    test_loader = DataLoader(dataset=test_data, batch_size=1)

                    previous_loss=current_loss
                    test_time=test_time+trainnetwork(train_testmodel, train_loader,test_model_optimizer)
                    accepted, current_loss=check_threshhold(train_testmodel, test_loader, y_test)
                    loss_diff=abs(previous_loss-current_loss)


                #checks if loss difference is small
                if abs(loss_diff)<(THRESH_HOLD/BAD_LOSS_PASSES):
                    loss_difference_counter=loss_difference_counter+1
                    if loss_difference_counter>BAD_LOSS_PASSES:
                        infinite_time_counter=infinite_time_counter+1
                        #print("Test model beaten. Total beaten by prospective ground truth: ", infinite_time_counter)
                        Model_beaten=True

            
                if Model_beaten:
                    break

            time_list.append(test_time)
            total_time=total_time+test_time

    
    
        #print(f'Iteration: {interationcounter} | Ground truth number: {i+1} | Average time:  {total_time/10:.3}')



        
           
    

    max_new_time=total_time/10
    time_for_graph.append(max_new_time)      

    
    
    #prints iteration data
    print(f'Iteration {interationcounter}: | Previous time: {current_time:.3f} | Best new time: {max_new_time:.3f} | Difference: {max_new_time-current_time:.3f} | # beaten: {infinite_time_counter}')
    interationcounter=interationcounter+1
    
    if infinite_time_counter==10:
        print("Potential unbreakable found")
        possible_infinite_time=prospective_GroundTruth.get_copy()
        print("Time: ", time_list)
        break
    
     
    
    
    time_increase=max_new_time-current_time
    
    
    
    #changes ground truth if better time found
    if max_new_time>current_time and max_new_time<current_time*1000:
        GroundTruth=model_list[0].get_copy()
        time_increase=True
        previous_time=current_time
        current_time=max_new_time
        no_increasecounter=0
        
    else:
        no_increasecounter=no_increasecounter+1

    #if no better ground truth found for 10 iterations, stops code                 
    if no_increasecounter==NO_TIME_INCREASES_STOP:
        print("No increase for ", NO_TIME_INCREASES_STOP,  " iterations")
        time_increase=False
 
      



First time:  0.07125449180603027
Iteration 1: | Previous time: 0.071 | Best new time: 0.402 | Difference: 0.331 | # beaten: 0
Iteration 2: | Previous time: 0.402 | Best new time: 2.471 | Difference: 2.069 | # beaten: 0
Iteration 3: | Previous time: 2.471 | Best new time: 0.524 | Difference: -1.947 | # beaten: 0
Iteration 4: | Previous time: 2.471 | Best new time: 1.215 | Difference: -1.256 | # beaten: 0
Iteration 5: | Previous time: 2.471 | Best new time: 4.712 | Difference: 2.241 | # beaten: 0
Iteration 6: | Previous time: 4.712 | Best new time: 1.578 | Difference: -3.134 | # beaten: 0
Iteration 7: | Previous time: 4.712 | Best new time: 2.390 | Difference: -2.322 | # beaten: 0
Iteration 8: | Previous time: 4.712 | Best new time: 0.942 | Difference: -3.770 | # beaten: 0
Iteration 9: | Previous time: 4.712 | Best new time: 1.690 | Difference: -3.022 | # beaten: 0
Iteration 10: | Previous time: 4.712 | Best new time: 1.454 | Difference: -3.258 | # beaten: 0
Iteration 11: | Previous time

Iteration 87: | Previous time: 36.763 | Best new time: 35.427 | Difference: -1.335 | # beaten: 0
Iteration 88: | Previous time: 36.763 | Best new time: 39.880 | Difference: 3.117 | # beaten: 0
Iteration 89: | Previous time: 39.880 | Best new time: 46.556 | Difference: 6.676 | # beaten: 0
Iteration 90: | Previous time: 46.556 | Best new time: 40.342 | Difference: -6.214 | # beaten: 0
Iteration 91: | Previous time: 46.556 | Best new time: 48.470 | Difference: 1.914 | # beaten: 0
Iteration 92: | Previous time: 48.470 | Best new time: 43.974 | Difference: -4.496 | # beaten: 0
Iteration 93: | Previous time: 48.470 | Best new time: 45.427 | Difference: -3.042 | # beaten: 0
Iteration 94: | Previous time: 48.470 | Best new time: 42.073 | Difference: -6.397 | # beaten: 0
Iteration 95: | Previous time: 48.470 | Best new time: 48.231 | Difference: -0.239 | # beaten: 0
Iteration 96: | Previous time: 48.470 | Best new time: 48.905 | Difference: 0.435 | # beaten: 0
Iteration 97: | Previous time: 48.

In [ ]:
#5 ground truth generated
time_for_graph=[]
GroundTruth = arbitraryLayerModel(GROUND_TRUTH_LAYERS)
#GroundTruth=possible_infinite_time.get_copy()
possible_infinite_time=arbitraryLayerModel(GROUND_TRUTH_LAYERS)



firstmodeltrained=arbitraryLayerModel(TESTMODULE_LAYERS)
test_model_optimizer = optim.SGD(firstmodeltrained.parameters(), lr=LEARNING_RATE)
accepted=False
current_time=0
data_list=[]


while((not accepted)):
    X, y=getdata_XY()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
    test_data = testData(torch.FloatTensor(X_test))
    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)                      
    test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)
    data_list.append(X)
    current_time=current_time+trainnetwork(firstmodeltrained, train_loader,test_model_optimizer)
    accepted=check_threshhold(firstmodeltrained, test_loader, y_test)
    
print("First time: ", current_time)
    
#iterates
interationcounter=1
#number of iterates in a row with no time increase
no_increasecounter=0
time_increase=True
#number of training interations where loss doesnt change
loss_difference_counter=0
stop_itterations=False
while(time_increase):
    node_list=[]
    time_list=[]
    model_list=[]
    for i in range(0, 10):
        node_list.append(select_node())
    
    test_model_list=[arbitraryLayerModel(TESTMODULE_LAYERS) for i in range(10)]
    for i in range(0, 5):
        total_time=0
        infinite_time_counter=0
        prospective_GroundTruth=GroundTruth.change_node_model(node_list[i])

        model_list.append(prospective_GroundTruth)

        for j, test_model in enumerate(test_model_list):
            current_loss=0
            previous_loss=0
            train_testmodel=test_model.get_copy()
            test_model_optimizer = optim.SGD(train_testmodel.parameters(), lr=LEARNING_RATE)
            data_counter=0
            test_time=0
            inital_data_length=len(data_list)
            accepted=False
            loss_difference_counter=0
            Model_beaten=False

            while(not accepted): 
                #use previously generated data
                if data_counter<inital_data_length:
                    X=data_list[data_counter]
                    y=get_yvals(prospective_GroundTruth, X)
                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                    test_data = testData(torch.FloatTensor(X_test))
                    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                    test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)




                    test_time=test_time+trainnetwork(train_testmodel, train_loader,test_model_optimizer)
                    previous_loss=current_loss
                    accepted, current_loss=check_threshhold(train_testmodel, test_loader, y_test)
                    loss_diff=abs(previous_loss-current_loss)
                    data_counter=data_counter+1
                #generate new data if needed
                else:                          
                    X=get_Xvals()
                    data_list.append(X)
                    y=get_yvals(prospective_GroundTruth, X)



                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                    test_data = testData(torch.FloatTensor(X_test))
                    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                    test_loader = DataLoader(dataset=test_data, batch_size=1)

                    previous_loss=current_loss
                    test_time=test_time+trainnetwork(train_testmodel, train_loader,test_model_optimizer)
                    accepted, current_loss=check_threshhold(train_testmodel, test_loader, y_test)
                    loss_diff=previous_loss-current_loss


                #checks if loss difference is small
                
                if abs(current_loss)<(THRESH_HOLD/10):
                    loss_difference_counter=loss_difference_counter+1
                    if loss_difference_counter>BAD_LOSS_PASSES:
                        infinite_time_counter=infinite_time_counter+1
                        #print("Test model beaten. Total beaten by prospective ground truth: ", infinite_time_counter)
                        Model_beaten=True

            
                if Model_beaten:
                    break

            total_time=total_time+test_time

        time_list.append(test_time/10)

    
        #print(f'Iteration: {interationcounter} | Ground truth number: {i+1} | Average time:  {total_time/10:.3}')



        
           
    

    max_new_time=max(time_list)
    time_for_graph.append(max_new_time)      

    
    
    
    #prints iteration data
    print(f'Iteration {interationcounter}: | Previous time: {current_time:.3f} | Best new time: {max_new_time:.3f} | Difference: {max_new_time-current_time:.3f} | # beaten: {infinite_time_counter}')
    interationcounter=interationcounter+1
    
    if infinite_time_counter==10:
        print("Potential unbreakable found")
        possible_infinite_time=prospective_GroundTruth.get_copy()
        print("Time: ", time_list)
        break
    
     
    
    
    time_increase=max_new_time-current_time
    
    
    
    #changes ground truth if better time found
    if max_new_time>current_time:
        if LINCOMBO:
            GroundTruth=get_new_GroundTruth_lincombo(model_list, time_list, current_time)
        else:
            GroundTruth=model_list[time_list.index(max_new_time)].get_copy()
        time_increase=True
        previous_time=current_time
        current_time=max_new_time
        no_increasecounter=0
        
    else:
        no_increasecounter=no_increasecounter+1

    #if no better ground truth found for 10 iterations, stops code                 
    if no_increasecounter==NO_TIME_INCREASES_STOP:
        print("No increase for ", NO_TIME_INCREASES_STOP,  " iterations")
        time_increase=False
 
      


In [3]:
#sparse traing and nonsparse test
GroundTruth
firstmodeltrained
sparsedata=(np.random.uniform( size=(10000, INPUTS))-0.5)*np.sqrt(12)
for i in sparsedata:
    for j in i:
        if np.random.randint(0,2)==0:
                j=0
for i in sparsedata:
    for j in i:
        if np.random.randint(0,2)==0:
            j=0
X=sparsedata
testmodel1=arbitraryLayerModel(TESTMODULE_LAYERS)
testmodel2=testmodel1.get_copy()

yvals1=get_yvals(GroundTruth, X)
yvals2=get_yvals(firstmodeltrained, X)



#"complicated mode"
X_train, X_test, y_train, y_test = train_test_split(X, yvals1, test_size=0.2, shuffle=False)
train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
test_data = testData(torch.FloatTensor(X_test))
train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
test_loader = DataLoader(dataset=test_data, batch_size=1)


test_model_optimizer = optim.SGD(testmodel1.parameters(), lr=LEARNING_RATE)
trainnetwork(testmodel1, train_loader,test_model_optimizer)
print(check_threshhold(testmodel1, test_loader, y_test))


X_train, X_test, y_train, y_test = train_test_split(X, yvals2, test_size=0.2, shuffle=False)
train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
test_data = testData(torch.FloatTensor(X_test))
train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
test_loader = DataLoader(dataset=test_data, batch_size=1)


test_model_optimizer = optim.SGD(testmodel2.parameters(), lr=LEARNING_RATE)
trainnetwork(testmodel2, train_loader,test_model_optimizer)
print(check_threshhold(testmodel2, test_loader, y_test))


X=(np.random.uniform( size=(3000, INPUTS))-0.5)*np.sqrt(12)
yvalstrue1=get_yvals(GroundTruth, X)
yvalstrue2=get_yvals(firstmodeltrained, X)
yvalstest1=get_yvals(testmodel1, X)
yvalstest2=get_yvals(testmodel2, X)
MSE1=mean_squared_error(yvalstrue1,yvalstest1)
MSE2=mean_squared_error(yvalstrue2,yvalstest2)
print("MSE complicated:", MSE1)
print("MSE simple:", MSE2)


(False, 0.058376869627523933)
(True, 0.0018080031886891534)
MSE complicated: 0.05548380551455631
MSE simple: 0.007346099797614052


In [11]:

print (GroundTruth.layer_input.weight)
print( GroundTruth.layer_input.bias)
for i in GroundTruth.layers:
    print( i.weight)
    print(i.bias)
    
print(GroundTruth.layer_output.weight)

Parameter containing:
tensor([[ 0.1359, -0.0980,  0.1708, -0.1607,  0.1386,  0.1629,  0.1406,  0.0505,
         -0.1587,  0.0831,  0.1777, -0.0885,  0.1646, -0.1003, -0.0296, -0.1726,
         -0.1718, -0.0535, -0.1002,  0.1812, -0.0720, -0.1062,  0.1453, -0.0404,
         -0.1733, -0.0300, -0.0199, -0.0550, -0.1297,  0.1176],
        [ 0.0458,  0.1796,  0.0523,  0.0985, -0.0116,  0.1490,  0.0078,  0.0207,
          0.0592, -0.0859, -0.1253,  0.0776,  0.1432, -0.0685, -0.1460,  0.1312,
          0.0283,  0.0471, -0.0740, -0.1787, -0.1051, -0.0838, -0.0346,  0.1701,
         -0.1595,  0.1209, -0.1021,  0.1444,  0.0218,  0.1164],
        [-0.0501,  0.0898, -0.1516, -0.0491,  0.1051, -0.0820,  0.1485, -0.0166,
         -0.1529, -0.1575,  0.1364,  0.1046, -0.1079, -0.0648, -0.0180, -0.0529,
          0.1368,  0.0062, -0.0791,  0.0543, -0.1450,  0.1026,  0.1222, -0.0453,
         -0.0975,  0.0921,  0.0221, -0.0216, -0.0145, -0.0053],
        [-0.0581, -0.0645, -0.1572, -0.1192,  0.0607,  0.

Parameter containing:
tensor([ 0.1625, -0.1505, -0.1725,  0.0062,  0.0719,  0.1361, -0.1395,  0.1452,
        -0.1132, -0.0388,  0.0613,  0.0472,  0.0355, -0.0832,  0.0069, -0.1740,
        -0.1298, -0.1618,  0.1160,  0.0938,  0.0077, -0.0869,  0.0322,  0.1617,
         0.0305,  0.1494,  0.0851, -0.1743, -0.0116, -0.0665],
       requires_grad=True)
Parameter containing:
tensor([[ 0.1013, -0.1493, -0.0242,  0.0470, -0.0684,  0.0039, -0.0976, -0.1611,
         -0.1810, -0.0923,  0.1624,  0.0620, -0.0790,  0.0162, -0.0805, -0.1302,
         -0.0554, -0.1580,  0.1698, -0.0985, -0.0466,  0.0510, -0.0288,  0.1654,
         -0.0998,  0.1025, -0.0409,  0.0127, -0.1160, -0.1070],
        [ 0.0851,  0.0288, -0.0426,  0.0776, -0.0150,  0.1763,  0.0835,  0.0724,
         -0.1502,  0.0178, -0.0985,  0.1781, -0.1104, -0.0404,  0.1509,  0.0533,
         -0.0620,  0.1363,  0.0669, -0.1562,  0.1195,  0.1015, -0.1142, -0.0543,
          0.1464, -0.0683,  0.0152, -0.0997,  0.0674, -0.0026],
        [-0.

In [14]:
complexmodel= torch.save(GroundTruth, "complex_weights.pth")

In [15]:
complexmodelcopy=GroundTruth.get_copy()

In [19]:
GroundTruth=complexmodelcopy.get_copy()